# Inception v3

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.utils import plot_model

# Load full InceptionV3 model
model = InceptionV3(
    weights='imagenet',       # Use pretrained ImageNet weights
    include_top=True,         # Include fully connected classification head
    input_shape=(224, 224, 3) # Standard input shape for ImageNet models
)

plot_model(
    model,
    to_file='inceptionv3_architecture.png',
    show_shapes=True,
    show_layer_names=False,
    rankdir='TB',
    expand_nested=False,
    dpi=70
)

In [ ]:
!apt-get install graphviz -y

In [ ]:
# Now you can uncomment the plot_model call in the Inception v3 cell.

# Resnet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import plot_model

# Load full ResNet50 model
model = ResNet50(
    weights='imagenet',       # Use pretrained ImageNet weights
    include_top=True,         # Include fully connected classification head
    input_shape=(224, 224, 3) # Standard input shape for ImageNet models
)

# Display model summary
model.summary()

# Print total parameters
print("✅ Total Parameters:", model.count_params())



# VGG 19 Architecture

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import plot_model

# Load full VGG19 model
model = VGG19(
    weights='imagenet',          # Use pretrained ImageNet weights
    include_top=True,            # Include the fully connected classification head
    input_shape=(224, 224, 3),   # Standard VGG input size
    classes=1000                 # 1000-class ImageNet output
)

# Print model summary
model.summary()

# Generate compact architecture diagram
plot_model(
    model,
    to_file='vgg19_small_architecture.png',  # Output image filename
    show_shapes=True,                        # Show layer input/output shapes
    show_layer_names=False,                  # Hide layer names to reduce clutter
    rankdir='TB',                            # Top-to-bottom layout
    expand_nested=False,                     # Don't expand nested layers
    dpi=70                                   # Lower DPI for smaller, compact image
)


# VGG 16 Code

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0       # Normalize pixel values to [0,1] for faster convergence

# --------------------------------------------------------------------------
# 6️⃣ Data Augmentation → Improves generalization and prevents overfitting
# --------------------------------------------------------------------------
datagen = ImageDataGenerator(
    rotation_range=10,        # Randomly rotate images by up to 10 degrees
    width_shift_range=0.1,    # Randomly shift images horizontally by 10% of width
    height_shift_range=0.1,   # Randomly shift images vertically by 10% of height
    horizontal_flip=True      # Randomly flip images horizontally
)
datagen.fit(x_train)          # Compute internal stats for normalization & augmentation

# --------------------------------------------------------------------------
# 1️⃣ Transfer Learning → Use pretrained VGG16 (on ImageNet) for feature extraction
# --------------------------------------------------------------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# 'weights=imagenet' loads pretrained filters learned on ImageNet (helps faster convergence)
# 'include_top=False' removes the original 1000-class classifier layer
# input_shape=(32,32,3) matches CIFAR-10 image size

# --------------------------------------------------------------------------
# 2️⃣ Freeze most layers → Prevents catastrophic forgetting of pretrained features
# 3️⃣ Fine-tune last 3–4 convolution blocks → Adapt high-level features to CIFAR-10
# --------------------------------------------------------------------------
for layer in base_model.layers[:-4]:    # Freeze all layers except the last 4
    layer.trainable = False             # These frozen layers retain pretrained weights

# --------------------------------------------------------------------------
# Add new classification head (custom top layers)
# --------------------------------------------------------------------------
model = models.Sequential([
    base_model,                                         # Pretrained convolutional base
    layers.Flatten(),                                   # Flatten feature maps to 1D vector
    layers.Dense(512, activation='relu'),               # Dense layer learns new CIFAR-10-specific features
    layers.Dropout(0.5),                                # 4️⃣ Dropout → randomly deactivate 50% neurons (prevents overfitting)
    layers.Dense(10, activation='softmax')              # Output layer for 10 CIFAR-10 classes
])

# --------------------------------------------------------------------------
# 5️⃣ Small Learning Rate (1e-4) → Stable fine-tuning of pretrained weights
# --------------------------------------------------------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-4),                 # Adam optimizer with low LR for controlled updates
    loss='sparse_categorical_crossentropy',             # Suitable for integer-labeled multi-class classification
    metrics=['accuracy']                                # Track model accuracy during training
)

# --------------------------------------------------------------------------
# Train model → Only 20 epochs needed due to transfer learning
# --------------------------------------------------------------------------
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),      # Use augmented data batches
    validation_data=(x_test, y_test),                   # Validate on unseen test data each epoch
    epochs=20,                                          # Train for 20 epochs — enough to reach ~90% accuracy
    verbose=1                                           # Show detailed training output
)

# --------------------------------------------------------------------------
# Evaluate model performance on test set
# --------------------------------------------------------------------------
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)  # Compute final loss & accuracy
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")                 # Display model accuracy


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0




# Data Augmentation (light but effective)
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)

# Load pretrained VGG16 (without top FC layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze lower convolutional layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add new custom classification head
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train (only ~20 epochs)
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    validation_data=(x_test, y_test),
    epochs=20,
    verbose=1
)

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0       # Normalize pixel values to [0,1] for faster convergence

# --------------------------------------------------------------------------
# Data Augmentation → Improves generalization and prevents overfitting
# --------------------------------------------------------------------------
datagen = ImageDataGenerator(
    rotation_range=10,        # Randomly rotate images by up to 10 degrees
    width_shift_range=0.1,    # Randomly shift images horizontally by 10% of width
    height_shift_range=0.1,   # Randomly shift images vertically by 10% of height
    horizontal_flip=True      # Randomly flip images horizontally
)
datagen.fit(x_train)          # Compute internal stats for normalization & augmentation

# --------------------------------------------------------------------------
# 1️⃣ Transfer Learning → Use pretrained VGG16 (on ImageNet) for feature extraction
# --------------------------------------------------------------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
# 'weights=imagenet' loads pretrained filters learned on ImageNet (helps faster convergence)
# 'include_top=False' removes the original 1000-class classifier layer
# input_shape=(32,32,3) matches CIFAR-10 image size

# --------------------------------------------------------------------------
# 2️⃣ Freeze most layers → Prevents catastrophic forgetting of pretrained features
# 3️⃣ Fine-tune last 3–4 convolution blocks → Adapt high-level features to CIFAR-10
# --------------------------------------------------------------------------
for layer in base_model.layers[:-4]:    # Freeze all layers except the last 4
    layer.trainable = False             # These frozen layers retain pretrained weights

# --------------------------------------------------------------------------
# Add new classification head (custom top layers)
# --------------------------------------------------------------------------
model = models.Sequential([
    base_model,                                         # Pretrained convolutional base
    layers.Flatten(),                                   # Flatten feature maps to 1D vector
    layers.Dense(512, activation='relu'),               # Dense layer learns new CIFAR-10-specific features
    layers.Dropout(0.5),                                # 4️⃣ Dropout → randomly deactivate 50% neurons (prevents overfitting)
    layers.Dense(10, activation='softmax')              # Output layer for 10 CIFAR-10 classes
])

# --------------------------------------------------------------------------
# 5️⃣ Small Learning Rate (1e-4) → Stable fine-tuning of pretrained weights
# --------------------------------------------------------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-4),                 # Adam optimizer with low LR for controlled updates
    loss='sparse_categorical_crossentropy',             # Suitable for integer-labeled multi-class classification
    metrics=['accuracy']                                # Track model accuracy during training
)

# --------------------------------------------------------------------------
# Train model → Only 20 epochs needed due to transfer learning
# --------------------------------------------------------------------------
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),      # Use augmented data batches
    validation_data=(x_test, y_test),                   # Validate on unseen test data each epoch
    epochs=20,                                          # Train for 20 epochs — enough to reach ~90% accuracy
    verbose=1                                           # Show detailed training output
)

# --------------------------------------------------------------------------
# Evaluate model performance on test set
# --------------------------------------------------------------------------
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)  # Compute final loss & accuracy
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")                 # Display model accuracy


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import plot_model

# Load full VGG16 model
model = VGG16(
    weights='imagenet',
    include_top=True,
    input_shape=(224, 224, 3),
    classes=1000
)

model.summary()

# Generate compact architecture diagram
plot_model(
    model,
    to_file='vgg16_small_architecture.png',
    show_shapes=True,
    show_layer_names=False,      # Reduces clutter
    rankdir='TB',
    expand_nested=False,
    dpi=70                      # Lower DPI for smaller image
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import plotly.graph_objects as go

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels if needed
# y_train = tf.keras.utils.to_categorical(y_train, 10)
# y_test = tf.keras.utils.to_categorical(y_test, 10)

# VGG16-like architecture with 5 conv + 2 FC layers
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))  # 5th conv layer
model.add(layers.MaxPooling2D((2, 2), strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))  # Second ANN layer
model.add(layers.Dense(10, activation='softmax'))

# Show architecture and parameter count
model.summary()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train with custom epochs and batch size
history = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=10,            # Change as needed
                    batch_size=64)        # Change as needed

# Plot training/validation accuracy and loss using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Training Acc'))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validation Acc'))
fig.update_layout(title='Training & Validation Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Training Loss'))
fig2.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validation Loss'))
fig2.update_layout(title='Training & Validation Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig2.show()


# Python

In [ ]:
g = 12
type (g)
b = int(12)
c = 3.1415
type(c)

In [ ]:
S = "welcome"
S1 = 'welcome'
S3 = '''hello'''
S4 = """Hello python"""
id(S)

In [ ]:
l3 = [1, 2, [3,4,5], 6]
len(l3)
l3[2][1]
type(l3)
print(l3)

In [ ]:
print("enter first no")
x = int(input())
print("enter second no")
y = int(input())
z = x+y
print(z)

In [ ]:
t1 = tuple([1,2,3])
type(t1)



In [ ]:
t3=(1,)
type(t3)

In [ ]:
s1 = {1,2,3,4,5}
type(s1)

In [ ]:
# A set doesn't store duplicate values
s2 = {1,2,1,2,5,4}
print(s2)

In [ ]:
d1 = {1:1, 2:2, 3:5} # here 1,2,3 are keys and 1,2,5 are values
type(d1)

In [ ]:
def sum_list(l):
    size = len(l)
    r = 0
    for i in range(size):
        r = r + l[i]
    return r

l = [12, 23, 5, 9]
result = sum_list(l)
print("sum:", result)


In [ ]:
def product(l):
    p = 1
    size = len(l)
    for i in range(size):
        p *= l[i]
    return p

l = [1, 2, 3, 4]
result = product(l)
print("Result:", result)

In [ ]:
def meanlist(l):
    total = 0
    size = len(l)
    for i in range(size):
        total += l[i]
    return total / size

l = [1, 2, 3, 4]
result = meanlist(l)
print("Result:", result)

In [ ]:
print(20%6)

In [ ]:
# Class for Rectangle
class Rectangle:
    def __init__(self, l: float, b: float):
        self.length = l
        self.breadth = b

    def area(self) -> float:
        return self.length * self.breadth

    def perimeter(self) -> float:
        return 2 * (self.length + self.breadth)

    def getLength(self) -> float:
        return self.length

    def getBreadth(self) -> float:
        return self.breadth

    def setLength(self, l: float) -> None:
        self.length = l

    def setBreadth(self, b: float) -> None:
        self.breadth = b


r1 = Rectangle(4, 5)
print(f"Rectangle Area: {r1.area()}")
print(f"Rectangle Perimeter: {r1.perimeter()}")


# Class for Cuboid
class Cuboid:
    def __init__(self, l: float, b: float, h: float):
        self.length = l
        self.breadth = b
        self.height = h

    def volume(self) -> float:
        return self.length * self.breadth * self.height

    def surface_area(self) -> float:
        return 2 * (self.length*self.breadth + self.length*self.height + self.breadth*self.height)

    def getLength(self) -> float:
        return self.length

    def getBreadth(self) -> float:
        return self.breadth

    def getHeight(self) -> float:
        return self.height

    def setLength(self, l: float) -> None:
        self.length = l

    def setBreadth(self, b: float) -> None:
        self.breadth = b

    def setHeight(self, h: float) -> None:
        self.height = h

c1 = Cuboid(4, 5, 6)
print(f"Cuboid Volume: {c1.volume()}")
print(f"Cuboid Surface Area: {c1.surface_area()}")

#List

In [ ]:
a = [] #empty list
mylist = ["apple", "banana", "cherry"]
print(mylist)
print(len(mylist))

In [ ]:
a = [1,2,3]
a.append([4,5,6])
print(a)

In [ ]:
a = [1,2,3]
a.extend([4,5,6])
print(a)

In [ ]:
fruits = ['apple', 'banana', 'cherry']

fruits.reverse()
print(fruits)

In [ ]:
a = [3, 4, 5, 6, 7, 8, 9]
a = a[:5]
print(a)


#tuple

In [ ]:
thistuple = ("apple", "banana", "cherry")
print(thistuple)

In [ ]:
#length in tuple
thistuple = ("apple", "banana", "cherry")
print(len(thistuple))

#sets
##A set is a collection which is unordered, unchangeable*, and unindexed.

##Set items are unordered, unchangeable, and do not allow duplicate values.

In [ ]:
thisset = {"apple", "banana", "cherry"}
print(thisset)

In [ ]:
# Class for Square
class Square:
    def __init__(self, side: float):
        self.side = side

    def area(self) -> float:
        return self.side * self.side

    def perimeter(self) -> float:
        return 4 * self.side

    def getSide(self) -> float:
        return self.side

    def setSide(self, side: float) -> None:
        self.side = side


# Example usage
s1 = Square(5)
print(f"Square Side: {s1.getSide()}")
print(f"Square Area: {s1.area()}")
print(f"Square Perimeter: {s1.perimeter()}")

s1.setSide(8)  # changing side
print(f"\nAfter changing side to {s1.getSide()}:")
print(f"Square Area: {s1.area()}")
print(f"Square Perimeter: {s1.perimeter()}")

In [ ]:
# Class for Cube
class Cube:
    def __init__(self, side: float):
        self.side = side

    def volume(self) -> float:
        return self.side ** 3

    def surface_area(self) -> float:
        return 6 * (self.side ** 2)

    def getSide(self) -> float:
        return self.side

    def setSide(self, side: float) -> None:
        self.side = side


# Example usage
c1 = Cube(4)
print(f"Cube Side: {c1.getSide()}")
print(f"Cube Volume: {c1.volume()}")
print(f"Cube Surface Area: {c1.surface_area()}")

c1.setSide(7)  # updating side
print(f"\nAfter changing side to {c1.getSide()}:")
print(f"Cube Volume: {c1.volume()}")
print(f"Cube Surface Area: {c1.surface_area()}")

In [ ]:
# Example: Bank Account Class

class BankAccount:
    def __init__(self, account_number, holder_name, balance=0.0):
        self.account_number = account_number
        self.holder_name = holder_name
        self.balance = balance

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            print(f"Deposited ₹{amount}. New Balance = ₹{self.balance}")
        else:
            print("Deposit amount must be positive.")

    def withdraw(self, amount):
        if 0 < amount <= self.balance:
            self.balance -= amount
            print(f"Withdrew ₹{amount}. Remaining Balance = ₹{self.balance}")
        else:
            print("Insufficient balance or invalid amount.")

    def get_balance(self):
        return self.balance


# Example usage
acc1 = BankAccount("123456", "Anshuman", 5000)

print(f"Account Holder: {acc1.holder_name}, Balance: ₹{acc1.get_balance()}")

acc1.deposit(2000)
acc1.withdraw(3000)
acc1.withdraw(5000)   # should show insufficient balance

In [ ]:
# Example: Constructor and Destructor in Python

class Car:
    def __init__(self, brand, model):   # Constructor
        self.brand = brand
        self.model = model
        print(f"Car {self.brand} {self.model} is created.")

    def display(self):
        print(f"Car Brand: {self.brand}, Model: {self.model}")

    def __del__(self):   # Destructor
        print(f"Car {self.brand} {self.model} is destroyed.")


# Creating object
c1 = Car("Tesla", "Model S")
c1.display()

# Deleting object manually
del c1

print("End of Program")

In [ ]:
# Example: Encapsulation in Python

class BankAccount:
    def __init__(self, account_number, holder_name, balance=0.0):
        self.__account_number = account_number   # private attribute
        self.__holder_name = holder_name         # private attribute
        self.__balance = balance                 # private attribute

    # Getter for balance
    def get_balance(self):
        return self.__balance

    # Setter for deposit
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            print(f"Deposited ₹{amount}. New Balance = ₹{self.__balance}")
        else:
            print("Deposit amount must be positive.")

    # Setter for withdrawal
    def withdraw(self, amount):
        if 0 < amount <= self.__balance:
            self.__balance -= amount
            print(f"Withdrew ₹{amount}. Remaining Balance = ₹{self.__balance}")
        else:
            print("Insufficient balance or invalid amount.")


# Example usage
acc = BankAccount("12345", "Anshuman", 5000)

# Access via getters/setters ✅
print("Initial Balance:", acc.get_balance())
acc.deposit(2000)
acc.withdraw(3000)

In [ ]:
from abc import ABC, abstractmethod

# Abstract Class
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass   # must be implemented in child class

    @abstractmethod
    def perimeter(self):
        pass   # must be implemented in child class


# Concrete Class 1
class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius * self.radius

    def perimeter(self):
        return 2 * 3.14 * self.radius


# Concrete Class 2
class Rectangle(Shape):
    def __init__(self, length, breadth):
        self.length = length
        self.breadth = breadth

    def area(self):
        return self.length * self.breadth

    def perimeter(self):
        return 2 * (self.length + self.breadth)


# Example usage
c = Circle(5)
print("Circle -> Area:", c.area(), ", Perimeter:", c.perimeter())

r = Rectangle(4, 6)
print("Rectangle -> Area:", r.area(), ", Perimeter:", r.perimeter())

In [ ]:
def my_average(l):
    total = 0
    size = len(l)
    for i in range(size):
        total += l[i]
    avg = total / size
    return avg

l = [1, 2, 3, 4]
result = my_average(l)
print("Average:", result)


In [ ]:
# 1. Function with argument and return value
def sum(a, b):
    return a + b

result = sum(5, 3)

# 2. Function with argument and no return value
def sum(a, b):
    print(f"Sum: {a + b}")

sum(5, 3)

# 3. Function with no argument and return value
def sum():
    a = 10
    b = 20
    return a + b

result = sum()

# 4. Function with no argument and no return value
def sum():
    a = 10
    b = 30
    print(f"Sum: {a + b}")

sum()


In [ ]:
# 1. Default Parameter Function
def sum_default(a=0, b=20):
    # This function takes two parameters, with b having a default value of 20.
    # If no parameter is passed, default values are used.
    return a + b

# Example usages:
a = sum_default(20, 10)  # Output: 30
b = sum_default(30)      # Output: 50 (b uses default value 20)
c = sum_default()        # Output: 20 (both a and b use defaults)


# 2. Named (Keyword) Parameter Function
def sum_named(a=15, b=50):
    # Parameters can be specified using keywords for clarity.
    return a + b

# Example usages:
d = sum_named(a=15, b=50)      # Output: 65 (explicitly assigned)
e = sum_named(b=20, a=50)      # Output: 70 (order doesn't matter with keyword arguments)
f = sum_named(b=70)            # Output: 85 (a uses default, b overridden)


# 3. Variable Number of Arguments Function
def sum_varargs(*args):
    # This function takes a variable number of arguments & returns their sum
    total = 0
    for i in args:
        total += i
    return total

# Example
a = sum_varargs(30)           # Output: 30
b = sum_varargs(1, 30, 36, 40) # Output: 107


# 4. Recursive Function (Factorial Example)
def factorial(n):
    # This function computes factorial of n recursively.
    # It's a direct recursive function.
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1) # Calls itself

# Example
fact = factorial(5)  # Output: 120


# 5. Notes on Recursive Types (for reference, not code)
# Types of recursive functions:
#   - Direct Recursive: Function calls itself inside its own definition.
#   - Indirect Recursive: Function calls another function, which eventually calls the first function.
#   - Nontail Recursive: The recursive call is not the last operation in the function.
#   - Tail Recursive: The recursive call is the last operation (Python does not optimize for tail recursion).


In [ ]:
# Indirect Recursive Functions
def fun1(n):
    """
    Indirect recursive function.
    Calls fun2 instead of itself if n > 10.
    """
    if n > 10:
        # Calls fun2, indirectly causing recursion
        return fun2(n + 1)
    # Base case: when n <= 10, stop recursion
    return n

def fun2(n):
    """
    Indirect recursive function.
    Calls fun1 instead of itself if n > 10.
    """
    if n > 10:
        # Calls fun1, continuing the indirect recursion
        return fun1(n + 1)
    # Base case: when n <= 10, stop recursion
    return n

# Example usage:
result = fun1(9)  # Starts at fun1, then indirect recursion happens until n <= 10
print(result)     # Output: 11


In [ ]:
def print_1_to_n(n):
    if n < 1:
        return
    print_1_to_n(n - 1)  # recursive call before print (non tail recursive)
    print(n, end=' ')    # print after recursion (ascending order)

def print_n_to_1(n):
    if n < 1:
        return
    print(n, end=' ')    # print before recursion (tail recursive)
    print_n_to_1(n - 1)  # tail recursion (descending order)

n = 5
print("Print from 1 to n:")
print_1_to_n(n)          # Output: 1 2 3 4 5

print("\nPrint from n to 1:")
print_n_to_1(n)          # Output: 5 4 3 2 1


In [ ]:
# Anonymous Function (Lambda): A small, unnamed function defined with the lambda keyword
# Used for short, simple functions without formally defining with def
square = lambda x: x * x            # anonymous function to square a number
print(square(5))                   # Output: 25

# Keyword Argument: Function arguments passed explicitly by name when calling
# Allows passing arguments in any order and improves readability
def greet(name, message="Hello"):
    print(f"{message}, {name}!")  # prints greeting with message and name

greet(name="Alice", message="Hi") # Output: Hi, Alice!
greet(message="Welcome", name="Bob") # order doesn't matter

# Higher Order Function: Function that takes another function as argument or returns a function
# Enables functional programming techniques, like passing behavior as parameters
def apply_func(func, value):
    return func(value)             # calls passed function on value

def increment(x):
    return x + 1

print(apply_func(increment, 7))   # Output: 8


In [ ]:
# Example for dir()
# dir() returns a list of names in the current local scope.
# If called with an argument, it attempts to return a list of valid attributes for that object.
print("dir() example:")
print(dir(__builtins__)) # Shows attributes of the built-in module
print("-" * 20)

# Example for id()
# id() returns the identity of an object. This is an integer which is guaranteed to be unique and constant for this object during its lifetime.
x = [1, 2, 3]
y = [1, 2, 3]
z = x
print("id() example:")
print(f"id(x): {id(x)}")
print(f"id(y): {id(y)}") # Different object, different id
print(f"id(z): {id(z)}") # Same object as x, same id
print("-" * 20)

# Example for open(file, mode)
# open() opens a file and returns a file object.
# The mode argument specifies how the file is to be opened (e.g., 'r' for read, 'w' for write, 'a' for append).
# We'll create a dummy file for this example.
file_content = "Hello, this is a test file.\nIt has two lines."
with open("test_file.txt", "w") as f:
    f.write(file_content)

print("open() example (reading from 'test_file.txt'):")
try:
    with open("test_file.txt", "r") as f:
        content = f.read()
        print(content)
except FileNotFoundError:
    print("test_file.txt not found.")
print("-" * 20)

# Example for all()
# all() returns True if all elements of an iterable are true (or if the iterable is empty).
list1 = [True, True, True]
list2 = [True, False, True]
list3 = []
print("all() example:")
print(f"all({list1}): {all(list1)}")
print(f"all({list2}): {all(list2)}")
print(f"all({list3}): {all(list3)}")
print("-" * 20)

# Example for len()
# len() returns the number of items in an object.
my_list = [10, 20, 30, 40, 50]
my_string = "Hello Python"
my_dict = {'a': 1, 'b': 2, 'c': 3}
print("len() example:")
print(f"Length of list {my_list}: {len(my_list)}")
print(f"Length of string '{my_string}': {len(my_string)}")
print(f"Length of dictionary {my_dict}: {len(my_dict)}")
print("-" * 20)

# Example for map()
# map() applies a given function to each item of an iterable and returns a map object (an iterator).
def square(x):
    return x * x

numbers = [1, 2, 3, 4, 5]
squared_numbers = map(square, numbers)
print("map() example (squaring numbers):")
print(f"Original numbers: {numbers}")
print(f"Squared numbers (using map): {list(squared_numbers)}") # Convert map object to list for printing
print("-" * 20)

#PANDAS

In [ ]:
import pandas as pd
df = pd.DataFrame()
print(df)

In [ ]:
import pandas as pd

lst = ['Geeks', 'For', 'Geeks', 'is',
            'portal', 'for', 'Geeks']

df = pd.DataFrame(lst)
print(df)

In [ ]:
import numpy as np
import pandas as pd

data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
df = pd.DataFrame(data, columns=['A', 'B', 'C'])
print(df)

In [ ]:
#creating dataframe from list of dictionaries
import pandas as pd

dict = {'name':["aparna", "anshu", "ayush", "dipanshu"],
        'degree': ["MBA", "BCA", "M.Tech", "MBA"],
        'score':[90, 40, 80, 98]}

df = pd.DataFrame(dict)

print(df)

In [ ]:
#different ways to create dataframe
import pandas as pd

# initialize data of lists.
data = {'Name': ['ash', 'nikhil', 'krish', 'jack'],
        'Age': [20, 21, 19, 18]}

# Create DataFrame
df = pd.DataFrame(data)

print(df)

In [ ]:
#creating dataframe from list of arrays
import pandas as pd

data = [['ayush', 10], ['anshu', 15], ['juli', 14]]

df = pd.DataFrame(data, columns=['Name', 'Age'])

print(df)

In [ ]:
#creating dataframe from list of dictionaries
import pandas as pd

data = [{'column a': 1, 'column b': 2, 'column c': 3},
        {'column a': 10, 'column b': 20, 'column c': 30}]

# Creates DataFrame.
df = pd.DataFrame(data)

print(df)

In [ ]:
original_df = pd.DataFrame({
    'Name': ['Anshu', 'Ayush', 'Dipanshu', 'Aparna'],
    'Age': [21, 21, 20, 21]
})

new_df = original_df[['Name']]
print(new_df)

In [ ]:
import pandas as pd

d = {'column one': pd.Series([10, 20, 30, 40],
                      index=['a', 'b', 'c', 'd']),
     'column two': pd.Series([10, 20, 30, 40],
                      index=['a', 'b', 'c', 'd'])}

df = pd.DataFrame(d)

print(df)

In [ ]:
#creating data frame using zip function
import pandas as pd
Name = ['Anshu', 'Dipanshu', 'Ayush', 'Aparna']
Age = [25, 30, 26, 22]

# get the list of tuples from two lists.
# and merge them by using zip().
list_of_tuples = list(zip(Name, Age))

list_of_tuples


# Converting lists of tuples into
# pandas Dataframe.
df = pd.DataFrame(list_of_tuples,
                  columns=['Name', 'Age'])

print(df)

In [ ]:
#Accessing and modification of Index
import pandas as pd

data = {'Name': ['John', 'Alice', 'Bob', 'Eve', 'Charlie'],
        'Age': [25, 30, 22, 35, 28],
        'Gender': ['Male', 'Female', 'Male', 'Female', 'Male'],
        'Salary': [50000, 55000, 40000, 70000, 48000]}

df = pd.DataFrame(data)
print(df.index)  # Accessing the index